In [ ]:
## Data collection - Scraping stock data

In [14]:
# This cell sets up the code to scrape the stock data from Naver Finance for SK Innovation(096770). The reason that SK Innovation was chosen was because I made over 50% of my initial amount by trading this stock over a short period, and I wanted to apply my ad-hoc logic to a systematic and reproducible method, hence this project.

import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Change the stock ticker to collect different stock data
stock_id = '096770'

def get_stocks(stock_id,max_limit):
    cate = []
    stocks = [] 

    count = 1

    while count < max_limit:
        base_url = 'https://finance.naver.com/item/frgn.nhn?code='+str(stock_id)+'&page='+str(count)
        res = requests.get(base_url)
        html = BeautifulSoup(res.content,'html.parser')
        # Multiple tables of same name and class. Therefore use findAll and select the wanted table.
        table_all = html.findAll('table',{'class':'type2'})
        table_0 = table_all[1]
        cate_0 = table_0.find_all('th')
        main_0 = table_0.find_all('span')
        # span_0 = main_0.find('span')

        # For first iteration (count==1) append the headers to cate.
        if count == 1:
            for i in range(len(cate_0)):
                if (i == 5 or i== 6):
                    continue
                else:
                    cate.append(cate_0[i].text)

            date = []
            close = []
            change = []
            percentage_change = []
            volume = []
            org_volume = []
            foreign_volume = []
            foreign_count = []
            foreign_percentage = []

        for i in range(len(main_0)):

            if(i%9 == 0):
                date.append(main_0[i].text)
            if(i%9 == 1):
                close.append(main_0[i].text)
            if(i%9 == 2):
                change.append(main_0[i].text)
            if(i%9 == 3):
                percentage_change.append(main_0[i].text)
            if(i%9 == 4):
                volume.append(main_0[i].text)
            if(i%9 == 5):
                org_volume.append(main_0[i].text)
            if(i%9 == 6):
                foreign_volume.append(main_0[i].text)
            if(i%9 == 7):
                foreign_count.append(main_0[i].text)
            if(i%9 == 8):
                foreign_percentage.append(main_0[i].text)
          
        df_data = [date, close, change, percentage_change, volume, org_volume, foreign_volume, foreign_count, foreign_percentage]       
        # print(cate)

        count += 1
    df = pd.DataFrame(columns = cate)
    df_data = ['Date', 'Close', 'Change', 'Pct_Change', 'Volume', 'Org_Volume', 'Foreign_Volume', 'Foreign_Count', 'Foreign_Pct'] 
    df.columns = df_data
    # df['날짜'] = date
    # df['종가'] = close
    # df['전일비'] = change
    # df['등락률'] = percentage_change
    # df['거래량'] = volume
    # df['순매매량'] = org_volume
    # df['순매매량'] = foreign_volume
    # df['보유주수'] = foreign_count
    # df['보유율'] = foreign_percentage
    df['Date'] = date
    df['Close'] = close
    df['Change'] = change
    df['Pct_Change'] = percentage_change
    df['Volume'] = volume
    df['Org_Volume'] = org_volume
    df['Foreign_Volume'] = foreign_volume
    df['Foreign_Count'] = foreign_count
    df['Foreign_Pct'] = foreign_percentage
 
    # df['전일비'] = df['전일비'].map(lambda x: x.lstrip('\n\t').rstrip('\n\t'))
    df['Change'] = df['Change'].map(lambda x: x.lstrip('\n\t').rstrip('\n\t'))
    # df['등락률'] = df['등락률'].map(lambda x: x.lstrip('\n\t').rstrip('\n\t'))
    df['Pct_Change'] = df['Pct_Change'].map(lambda x: x.lstrip('\n\t').rstrip('\n\t'))

    
    return df 

In [39]:
## Data Cleaning


In [17]:
# This cell runs the scraper funciton above and scrape the data, and may take a (few) minute(s).

df = get_stocks(stock_id,71)

# Remove special characters in dataframe
df['Date'] = pd.to_datetime(df['Date'])
df = df.replace('\,','', regex=True)
df = df.replace('\+','', regex=True)
# df = df.replace('\-','', regex=True)
df = df.replace('\%','', regex=True)

# Set index as Date
df.set_index(df['Date'], inplace= True)
df.drop('Date',axis=1,inplace=True)

df.head()

,Close,Change,Pct_Change,Volume,Org_Volume,Foreign_Volume,Foreign_Count,Foreign_Pct
Date,,,,,,,,
2020-08-19,164000,3500,2.18,813043,55987,-24995,22325730,24.14
2020-08-18,160500,11500,-6.69,1703952,-42324,10852,22350725,24.17
2020-08-14,172000,10000,-5.49,1615501,-250971,36803,22339576,24.16
2020-08-13,182000,500,-0.27,945867,40746,-20077,22301981,24.12
2020-08-12,182500,4500,-2.41,1129184,-671,25353,22289063,24.11


In [18]:
# Fill missing values such as weekends/holidays
df = df.resample('D').asfreq()
df.sort_values(by=['Date'],ascending=False,inplace=True)
df.fillna(method='ffill',inplace=True)

In [19]:
stock_file = 'stock_'+ str(stock_id) +'.csv'
df.to_csv(str(stock_file))

In [20]:
## Tweak the KR_IR file

In [21]:
# Data from df_IR = pd.read_csv('./data/KR_IR.csv'). Code below to fill in missing dates, and forward fill missing values of IR to those missing dates, producing a complete dataframe of IR.

dates = pd.Index([pd.Timestamp('2015-03-12'), 
                  pd.Timestamp('2015-06-11'), 
                  pd.Timestamp('2016-06-09'), 
                  pd.Timestamp('2017-11-30'), 
                  pd.Timestamp('2018-11-30'), 
                  pd.Timestamp('2019-07-18'), 
                  pd.Timestamp('2019-10-16'), 
                  pd.Timestamp('2020-03-17'), 
                  pd.Timestamp('2020-05-28'), 
                  pd.Timestamp('2020-08-19')])
df_IR = pd.DataFrame([1.75,1.50,1.25,1.50,1.75,1.50,1.25,0.75,0.50,0.50], dates)
df_IR = df_IR.asfreq('D')
df_IR.fillna(method='ffill',inplace=True)
df_IR.reset_index(inplace=True)
df_IR.columns = ['Date','IR']
df_IR['Date'] = pd.to_datetime(df_IR['Date'])
df_IR.sort_values(by=['Date'],ascending=False,inplace=True)
df_IR.set_index(['Date'],inplace=True)

In [22]:
df_IR.head()

,IR
Date,
2020-08-19,0.5
2020-08-18,0.5
2020-08-17,0.5
2020-08-16,0.5
2020-08-15,0.5


In [23]:
## Clean DJI.csv

In [24]:
df_DJI = pd.read_csv('./data/DJI.csv')

df_DJI.info()
df_DJI.set_index(['Date'],inplace=True)
df_DJI.sort_values(by=['Date'],ascending=False,inplace=True)
drop_cols = ['Open', 'High','Low', 'Adj Close', 'Volume']

# Remove columns without relative significance.
df_DJI = df_DJI.drop(drop_cols,axis=1)
df_DJI.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1417 entries, 0 to 1416
Data columns (total 7 columns):
Date         1417 non-null object
Open         1417 non-null float64
High         1417 non-null float64
Low          1417 non-null float64
Close        1417 non-null float64
Adj Close    1417 non-null float64
Volume       1417 non-null int64
dtypes: float64(5), int64(1), object(1)
memory usage: 77.6+ KB


,Close
Date,
2020-08-18,27778.070313
2020-08-17,27844.910156
2020-08-14,27931.019531
2020-08-13,27896.720703
2020-08-12,27976.839844


In [25]:
df_DJI.index = pd.to_datetime(df_DJI.index)

df_DJI = df_DJI.resample('D').asfreq()
df_DJI.sort_values(by=['Date'],ascending=False,inplace=True)
df_DJI.fillna(method='ffill',inplace=True)

df_DJI.head()

,Close
Date,
2020-08-18,27778.070313
2020-08-17,27844.910156
2020-08-16,27844.910156
2020-08-15,27844.910156
2020-08-14,27931.019531


In [26]:
## Clean OIL_WTI.csv

In [27]:
df_WTI = pd.read_csv('./data/OIL_WTI.csv')

df_WTI.info()
df_WTI.set_index(['Date'],inplace=True)

df_WTI.index = pd.to_datetime(df_WTI.index)
df_WTI = df_WTI.resample('D').asfreq()

df_WTI.fillna(method='ffill',inplace=True)

df_WTI.sort_values(by=['Date'],ascending=False,inplace=True)

df_WTI.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1306 entries, 0 to 1305
Data columns (total 2 columns):
Date       1306 non-null object
USD_Bar    1306 non-null object
dtypes: object(2)
memory usage: 20.5+ KB


,USD_Bar
Date,
2020-08-10,41.94
2020-08-09,41.16
2020-08-08,41.16
2020-08-07,41.16
2020-08-06,41.93


In [28]:
## Clean USD_KRW_XR.csv 

In [29]:
df_XR = pd.read_csv('./data/USD_KRW_XR.csv')

df_XR.info()
df_XR.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 6 columns):
날짜      1470 non-null object
종가      1470 non-null object
오픈      1470 non-null object
고가      1470 non-null object
저가      1470 non-null object
변동 %    1470 non-null object
dtypes: object(6)
memory usage: 69.0+ KB


,날짜,종가,오픈,고가,저가,변동 %
0,2020년 08월 19일,"1,183.21","1,183.62","1,184.20","1,180.75",0.04%
1,2020년 08월 18일,"1,182.70","1,184.14","1,186.70","1,182.89",-0.04%
2,2020년 08월 17일,"1,183.19","1,185.90","1,187.27","1,183.63",-0.32%
3,2020년 08월 14일,"1,186.94","1,185.15","1,188.06","1,183.92",0.17%
4,2020년 08월 13일,"1,184.92","1,183.74","1,186.32","1,181.24",0.14%


In [30]:
drop_cols = ['오픈','고가','저가']
df_XR.drop(drop_cols,axis=1,inplace=True)

col_rename = ['Date','XR','Pct_Change']
df_XR.columns = col_rename

In [31]:
df_XR = df_XR.replace('년','-', regex=True)
df_XR = df_XR.replace('월','-', regex=True)
df_XR = df_XR.replace('일','', regex=True)
df_XR = df_XR.replace('\,','', regex=True)
df_XR = df_XR.replace('\%','', regex=True)
df_XR = df_XR.replace(' ','', regex=True)

df_XR.head()

,Date,XR,Pct_Change
0,2020-08-19,1183.21,0.04
1,2020-08-18,1182.70,-0.04
2,2020-08-17,1183.19,-0.32
3,2020-08-14,1186.94,0.17
4,2020-08-13,1184.92,0.14


In [32]:
df_XR['Date'] = pd.to_datetime(df_XR['Date'])
df_XR = df_XR.set_index('Date')

df_XR = df_XR.resample('D').asfreq()
df_XR.fillna(method='ffill',inplace=True)

df_XR.sort_values(by=['Date'],ascending=False,inplace=True)

df_XR.head()

,XR,Pct_Change
Date,,
2020-08-19,1183.21,0.04
2020-08-18,1182.70,-0.04
2020-08-17,1183.19,-0.32
2020-08-16,1186.94,0.17
2020-08-15,1186.94,0.17


In [33]:
df_STK = df

# All cleaned dataframes. But all dataframes have different shapes. Therefore, must unite into a single dataframe and order it by date.
df_DJI.shape
df_IR.shape
df_WTI.shape
df_XR.shape
df_STK.shape

(2081, 8)

In [34]:
# df_STK.tail()
start_date = '2015-08-10'
end_date = '2020-08-10'

df_DJI = df_DJI[(df_DJI.index >= start_date) & (df_DJI.index <= end_date)]
df_IR = df_IR[(df_IR.index >= start_date) & (df_IR.index <= end_date)]
df_WTI = df_WTI[(df_WTI.index >= start_date) & (df_WTI.index <= end_date)]
df_XR = df_XR[(df_XR.index >= start_date) & (df_XR.index <= end_date)]
df_STK = df_STK[(df_STK.index >= start_date) & (df_STK.index <= end_date)]

print('Shape of DJI: ', df_DJI.shape)
print('Shape of IR: ', df_IR.shape)
print('Shape of WTI: ', df_WTI.shape)
print('Shape of XR: ', df_XR.shape)
print('Shape of STK: ', df_STK.shape)



Shape of DJI:  (1828, 1)
Shape of IR:  (1828, 1)
Shape of WTI:  (1828, 1)
Shape of XR:  (1828, 2)
Shape of STK:  (1828, 8)


In [35]:
# Concat all dataframes into one

df_DJI.columns = ['DJI_Close']
df_XR.columns = ['XR','XR_Pct_Change']
df = pd.concat([df_STK, df_DJI,df_IR,df_WTI,df_XR], axis=1)

In [36]:
df.head()

,Close,Change,Pct_Change,Volume,Org_Volume,Foreign_Volume,Foreign_Count,Foreign_Pct,DJI_Close,IR,USD_Bar,XR,XR_Pct_Change
Date,,,,,,,,,,,,,
2020-08-10,186000,2000,1.09,3464467,-109960,-89376,22251502,24.06,27791.439453,0.5,41.94,1185.58,-0.30
2020-08-09,186000,2000,1.09,3464467,-109960,-89376,22251502,24.06,27791.439453,0.5,41.16,1189.18,0.44
2020-08-08,186000,2000,1.09,3464467,-109960,-89376,22251502,24.06,27791.439453,0.5,41.16,1189.18,0.44
2020-08-07,184000,20500,12.54,6000468,-112377,43102,22337710,24.16,27433.480469,0.5,41.16,1189.18,0.44
2020-08-06,163500,4500,2.83,4118060,122240,-139502,22293836,24.11,27386.980469,0.5,41.93,1184.00,-0.19


In [37]:
df.tail()

,Close,Change,Pct_Change,Volume,Org_Volume,Foreign_Volume,Foreign_Count,Foreign_Pct,DJI_Close,IR,USD_Bar,XR,XR_Pct_Change
Date,,,,,,,,,,,,,
2015-08-14,99100,200,0.20,410885,-19925,34641,35154341,38.02,17477.400391,1.5,42.45,1179.33,0.07
2015-08-13,98900,6100,6.57,766713,-30060,217401,35122235,37.98,17408.250000,1.5,42.27,1178.51,0.42
2015-08-12,92800,700,0.76,577495,-63720,158868,34906612,37.75,17402.509766,1.5,43.22,1173.58,-0.48
2015-08-11,92100,100,-0.11,550752,-36950,22190,34747744,37.58,17402.839844,1.5,43.11,1179.23,1.73
2015-08-10,92200,3800,-3.96,377629,-84907,-1962,34739524,37.57,17615.169922,1.5,44.94,1159.20,-0.47


In [38]:
df.to_csv('processed_data.csv')